In [142]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

TRADUCTOR_SIGFE_SIGCOM = pd.read_excel('input\\asociacion_sigfe_sigcom.xlsx')
TRADUCTOR_SIGFE_SIGCOM['Cod. SIGFE limpio'] = TRADUCTOR_SIGFE_SIGCOM['Cod. SIGFE limpio'].str.replace('.', '', regex = False)
TRADUCTOR_SIGFE_SIGCOM['Cod. SIGCOM'] = TRADUCTOR_SIGFE_SIGCOM['Cod. SIGCOM'].astype(int)
TRADUCTOR_SIGFE_SIGCOM = TRADUCTOR_SIGFE_SIGCOM.set_index('Cod. SIGFE limpio')

EXCEPCIONES_SIGFE = {
    '221299901601': 'FERNANDO BARAONA EN RRHH. ANEDIN EN GASTO GENERAL',
    '221299901602': 'Todo en RRHH',
    '221299900902': 'Cardiologia y Cardiocirugia en RRHH. UC Christus en gastos generales',
    '221299900201': 'M Meneses cargado en RRHH',
    '221299900202': 'J Andueza y Cardiocirugia en RRHH'
}

In [229]:
def desglosar_excepciones(agrupados_por_sigcom):
    df_devengos = pd.read_excel('input\\SA_DisponibilidadDevengoPresupuestario.xls', header = 5)
    df_devengos = df_devengos[['Titulo', 'Principal', 'Concepto Presupuestario', 'Monto Vigente', 'Monto Disponible', 'Monto Consumido']]
    df_devengos['Item Presupuestario'] = df_devengos['Concepto Presupuestario'].str.split() \
                                                                            .str[0]

    for concepto_pres_excepcion in EXCEPCIONES_SIGFE.keys():
        query_excepcion = df_devengos.query('`Item Presupuestario` == @concepto_pres_excepcion')

        if concepto_pres_excepcion == '221299901601':
            # FER BARAONA EN RRHH, todo lo demas en gastos generales
            mask_a_rrhh = query_excepcion['Principal'].str.contains('BARAONA')
        
        elif concepto_pres_excepcion == '221299901602':
            # Todo en RRHH
            mask_a_rrhh = query_excepcion['Principal'].notna()
        
        elif concepto_pres_excepcion == '221299900902':
            # CARDIOCIRUGUA Y CARDIOLOGIA EN RRHH, LO DEMAS EN GASTOS GENERALES.
            mask_a_rrhh = (query_excepcion['Principal'].str.contains('CARDIOLOGIA')) | \
                        (query_excepcion['Principal'].str.contains('CARDIOCIRUGIA'))

        elif concepto_pres_excepcion == '221299900201':
            # M MENESES EN RRHH, LO DEMAS EN GASTOS GENERALES
            mask_a_rrhh = query_excepcion['Principal'].str.contains('MANUEL MENESES')
            
        elif concepto_pres_excepcion == '221299900202':
            # ANDUEZA Y CARDIOCIRUGIA EN RRHH, LO DEMAS EN GASTOS GENERALES.
            mask_a_rrhh = query_excepcion['Principal'].str.contains('ANDUEZA') | \
                        query_excepcion['Principal'].str.contains('CARDIOCIRUGIA')
        
        df_a_rrhh = query_excepcion[mask_a_rrhh]
        df_a_gg = query_excepcion[~mask_a_rrhh]

        valor_a_rrhh = df_a_rrhh['Monto Vigente'].sum()
        valor_a_gg = df_a_gg['Monto Vigente'].sum()

        mask_excepcion = (agrupados_por_sigcom['Item Presupuestario'] == concepto_pres_excepcion)

        agrupados_por_sigcom.loc[mask_excepcion, 'Devengado_RRHH'] = valor_a_rrhh 
        agrupados_por_sigcom.loc[mask_excepcion, 'Devengado'] = valor_a_gg
    
    return agrupados_por_sigcom

In [230]:
def correr_programa():
    df = pd.read_excel('input\\SA_EstadoEjecucionPresupuestaria.xls', header = 6)
    df['Item Presupuestario'] = df['Concepto Presupuestario'].str.split() \
                                                             .str[0]
    df = df.set_index('Item Presupuestario')

    columnas_utiles = ['Devengado']
    agrupar_por = ['Cod. SIGCOM', 'Item en SIGCOM', 'Item Presupuestario']

    unidas = pd.merge(df, TRADUCTOR_SIGFE_SIGCOM, how = 'left', left_index = True, right_index = True)
    agrupados_por_presupuestario = unidas.groupby(agrupar_por, dropna = False)[columnas_utiles].sum().reset_index()
    agrupados_por_presupuestario = desglosar_excepciones(agrupados_por_presupuestario)

    agrupados_por_presupuestario['Cod. SIGCOM'] = agrupados_por_presupuestario['Cod. SIGCOM'].astype('Int32').astype(str)

    agrupados_por_sigcom = agrupados_por_presupuestario.groupby('Cod. SIGCOM').sum()

    return agrupados_por_presupuestario, agrupados_por_sigcom


In [238]:
gastos_sigfe, gastos_sigcom = correr_programa()

In [239]:
formato = pd.read_excel('input\\Formato 3_Gasto General 2022-09_2.xlsx')
formato = formato.set_index('Unnamed: 0')

columnas_antiguas = formato.columns
formato.columns = formato.columns.str.split('-').str[0]

for codigo_gasto in gastos_sigcom.index:
    valor_a_ingresar = gastos_sigcom.loc[codigo_gasto, 'Devengado']
    if codigo_gasto in formato.columns:
        formato.loc['Valor General', codigo_gasto] = valor_a_ingresar

formato.columns = columnas_antiguas
formato = formato.reset_index()

In [241]:
formato.to_excel('rellenado.xlsx', index = False)